<a href="https://colab.research.google.com/github/lerdeljan17/ML-Homework-1/blob/main/4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [ ]:
from nltk import FreqDist
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import regexp_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import numpy as np
import pandas as pd
import math
import re


class MultinomialNaiveBayes:
  def __init__(self, nb_classes, nb_words, pseudocount):
    self.nb_classes = nb_classes
    self.nb_words = nb_words
    self.pseudocount = pseudocount
  
  def fit(self, X, Y):
    nb_examples = X.shape[0]

    # Racunamo P(Klasa) - priors
    # np.bincount nam za datu listu vraca broj pojavljivanja svakog celog
    # broja u intervalu [0, maksimalni broj u listi]
    self.priors = np.bincount(Y) / nb_examples
    #print('Priors:')
    #print(self.priors)

    # Racunamo broj pojavljivanja svake reci u svakoj klasi
    occs = np.zeros((self.nb_classes, self.nb_words))
    for i in range(nb_examples):
      c = Y[i]
      for w in range(self.nb_words):
        cnt = X[i][w]
        occs[c][w] += cnt
    #print('Occurences:')
    #print(occs)
    
    # Racunamo P(Rec_i|Klasa) - likelihoods
    self.like = np.zeros((self.nb_classes, self.nb_words))
    for c in range(self.nb_classes):
      for w in range(self.nb_words):
        up = occs[c][w] + self.pseudocount
        down = np.sum(occs[c]) + self.nb_words*self.pseudocount
        self.like[c][w] = up / down
  #  print('Likelihoods:')
  #  print(self.like)
          
  def predict(self, bow):
    # Racunamo P(Klasa|bow) za svaku klasu
    probs = np.zeros(self.nb_classes)
    for c in range(self.nb_classes):
      prob = np.log(self.priors[c])
      for w in range(self.nb_words):
        cnt = bow[w]
        prob += cnt * np.log(self.like[c][w])
      probs[c] = prob
    # Trazimo klasu sa najvecom verovatnocom
   # print('\"Probabilites\" for a test BoW (with log):')
   # print(probs)
    prediction = np.argmax(probs)
    return prediction


# Korak 1: Učitavanje i obrada podataka.
filename = '/content/drive/My Drive/ML/ml_d1_x_y_z/data/twitter.csv'
all_data = pd.read_csv(filename,usecols=(1,2),encoding = "ISO-8859-1")
#print (all_data)
labels = all_data.drop(['SentimentText'], axis=1).values
#corpus = all_data.drop(['Sentiment'], axis=1).values
corpus = []
for s in all_data.drop(['Sentiment'], axis=1).values:
  corpus.extend(s)
#print(corpus[:150])
#print(labels)


# Nasumično mešanje.
nb_samples = len(corpus)
indices = np.random.permutation(nb_samples)
corpus = np.asarray(corpus)
corpus = corpus[indices]
labels = np.asarray(labels)
labels = labels[indices]

# Cistimo korpus
print('Cleaning the corpus...')
porter = PorterStemmer()
clean_corpus = []
stop_punc = set(stopwords.words('english')).union(set(punctuation))
for doc in corpus:
  #print(doc)
  words_remuname = re.sub(r'@[^\s]+', '', doc)
  #words_remlinks = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '', words_remuname)
  words = wordpunct_tokenize(words_remuname)
  #words = regexp_tokenize(doc, "[\w']+")
  words_lower = [w.lower() for w in words]
  #words_remuname = [re.sub(r'@[^\s]+', '', w) for w in words_lower]
  words_remspecial = [re.sub(r'[^a-zA-Z0-9]', '', w) for w in words_lower]
  #print(words_lower)
  words_filtered = [w for w in words_remspecial if w not in stop_punc]
  #print(words_filtered)
  words_stemmed = [porter.stem(w) for w in words_filtered]
  #print('Final:', words_stemmed)
  words_remempty = list(filter(None, words_stemmed))
  clean_corpus.append(words_remempty)
  #print()
  #print()
  # Primetiti razliku u vokabularu kada se koraci izostave
#print(clean_corpus[:150])

train_ratio = 0.8
nb_samples = len(clean_corpus)
nb_train = int(train_ratio * nb_samples)
data_train = dict()
train_corpus = clean_corpus[:nb_train]
train_labels = labels[:nb_train]
print (nb_train)
nb_test = nb_samples - nb_train
#data_test = dict()
test_corpus = clean_corpus[nb_train:]
test_labels = labels[nb_train:]
#print(train_corpus[:200])
# Kreiramo vokabular
#print('Creating the vocab...')
vocab_dict = dict()
'''
vocab_set = set()
for doc in train_corpus:
  for word in doc:
    #print(word)
    vocab_set.add(word)
vocab = list(vocab_set)
'''
for doc in train_corpus:
  #print(doc)
  for word in doc:
    vocab_dict.setdefault(word, 0)
    vocab_dict[word] += 1
    #print(word)
vocab = sorted(vocab_dict, key=vocab_dict.get, reverse=True)[:10000]

#print('Vocab:', list(zip(vocab, range(len(vocab)))))
#print('Feature vector size: ', len(vocab))

# 1: Bag of Words model sa 3 različita scoringa
np.set_printoptions(precision=2, linewidth=200)

def occ_score(word, doc):
   return 1 if word in doc else 0
  
def numocc_score(word, doc):
  return doc.count(word)

def freq_score(word, doc):
  return doc.count(word) / len(doc)

#BoW model
print('Creating BOW features...')
X = np.zeros((len(train_corpus), len(vocab)), dtype=np.float32)
for doc_idx in range(len(train_corpus)):
  doc = train_corpus[doc_idx]
  #print(doc)
  for word_idx in range(len(vocab)):
    word = vocab[word_idx]
    #print(word)
    #print()
    cnt = numocc_score(word, doc)
    #print(cnt)
    #print()
    X[doc_idx][word_idx] = cnt
    #if cnt > 1:
      #print(cnt,end='')
Y = np.zeros(len(train_corpus), dtype=np.int32)
for i in range(len(Y)):
  Y[i] = train_labels[i]

print('X:')
print(X)
'''
for f,i in enumerate(X):
  if f == 500:
    break
  for p in i:
    print(p)
'''
print('Y')
print(Y)
print()

def create_bow(doc, vocab):
	bow = np.zeros(len(vocab), dtype=np.float64)
	for word_idx in range(len(vocab)):
		word = vocab[word_idx]
		cnt = numocc_score(word, doc)
		bow[word_idx] = cnt
	return bow


def get_top_n(corpus, labels, n, label):
	cnt = 0
	count_dict = dict()
	for i in range(len(corpus)):
		if labels[i] == label:
			cnt += 1
			for word in corpus[i]:
				count_dict.setdefault(word, 0)
				count_dict[word] += 1

	return sorted(count_dict, key=count_dict.get, reverse=True)[:n]

def LR(corpus, labels, word):
  pos_count = 0
  neg_count = 0
  for i in range(len(corpus)):
	  doc = corpus[i]
	  if labels[i] == 1:
		  pos_count += doc.count(word)
	  else:
		  neg_count += doc.count(word)
    #print('pos =', pos_count, 'neg =', neg_count)
  return (pos_count / neg_count) if pos_count > 9 and neg_count > 9 else 0

Cleaning the corpus...
79991
Creating BOW features...
X:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
Y
[0 1 1 ... 0 1 1]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:

class_names = ['Negative', 'Positive']
model = MultinomialNaiveBayes(nb_classes=2, nb_words=len(vocab), pseudocount=1)
model.fit(X, Y)
#prediction = model.predict(test_bow)

match = 0
TP = 0
FP = 0
TN = 0
FN = 0
print()
print('Running tests...')
for i in range(len(test_corpus)):
  doc = test_corpus[i]
  #print(doc)
  #print()
  label = test_labels[i]
  
  bow = create_bow(doc, vocab)

  prediction = model.predict(bow)
  #print('Predicted class (with log): ', class_names[prediction])

  if prediction == test_labels[i]:
    match += 1

  if prediction == 1:
    if test_labels[i] == 1:
      TP += 1
    else:
      FP += 1
  else:
    if test_labels[i] == 0:
      TN += 1
    else:
      FN += 1

acc = match / len(test_corpus)

print(TP + FN + FP + TN)
conf_mat = [[TN, FP], [FN, TP]]

print('Accuracy = {}'.format(acc))
print('Confusion matrix', conf_mat)
print('Top 5 negative', get_top_n(clean_corpus, labels, 5, 0))
print('Top 5 positive', get_top_n(clean_corpus, labels, 5, 1))
best_LR_words = dict()
worst_LR_wordsP = dict()
for doc in clean_corpus:
  for word in doc:
    cnt = LR(clean_corpus,labels,word)
    best_LR_words.setdefault(word, 0)
    best_LR_words[word] = cnt
    #print(word)
best_LR_wordsP = sorted(best_LR_words, key=best_LR_words.get, reverse=True)[:5]
worst_LR_wordsP = sorted(best_LR_words, key=best_LR_words.get, reverse=False)[:5]


Running tests...
19998
Accuracy = 0.7454745474547455
Confusion matrix [[5973, 2742], [2348, 8935]]
Top 5 negative ['go', 'get', 'quot', 'like', 'miss']
Top 5 positive ['quot', 'thank', 'good', 'love', 'like']


In [ ]:
print('Best LR : ', best_LR_wordsP)
print('Worst LR : ', worst_LR_wordsP)

NameError: ignored